In [5]:
"""
this file will specify the config, and then call benchmark.py to calculate the brain score metrics.
this will also append the results to a common file.
"""
import sys
sys.path.append('/home/akazemi3/Desktoparchvision/archvision')
from omegaconf import OmegaConf


cfg_dict = {
    
# "wavelet": {
#     "layers": 1,
#     "type": "gabor",
# },
    
"conv": {
    "layers": 4,
    "layer_sizes": [None, 128, 256, 256],
    "kernel_sizes": [7, 5, 3, 3],
    "kernel_types":['curvature','random','random','random'],
    "pool_type":'max',
    "pool_kernel_sizes":[2,2,2,2],
    "init": "kaiming",
    "norm": "channel",
    "nonlin": "relu",
    },
    
"voxel_set": "OTC",
"eval_metric": "srpr",
"benchmark_data_type": "fMRI",
"log_expdata": True,  # flag to save the training data
"log_dir": "logs/",  # directory to save experimental data
"exp_name": "test",  # logs are stored under a directory created under this name
}

cfg = OmegaConf.create(cfg_dict)
config = OmegaConf.from_cli()
cfg = OmegaConf.merge(cfg, config)

In [8]:
import torch
import torchvision.transforms as transforms
from models.backbone import VisionModel

sys.path.append('/home/akazemi3/Desktop/DeepJuiceDev')
from deepjuice import * # the juice
from benchmarks.nsd_data import NSDBenchmark

devices = {'device': 'cuda', 'output_device': 'cpu'}


preprocess = transforms.Compose([
    transforms.Resize(256),             
    transforms.CenterCrop(224),         
    transforms.ToTensor(),             
    transforms.Normalize(             
        mean=[0.485, 0.456, 0.406],     
        std=[0.229, 0.224, 0.225])
])

model = VisionModel(cfg, device=devices['device'])
print(model(torch.Tensor(1,3,224,224)).shape) # test with sample image


benchmark = NSDBenchmark() #load brain data benchmark
dataloader = get_data_loader(benchmark.image_paths, preprocess)

extractor = FeatureExtractor(model, dataloader, flatten=True,
                             batch_progress=True, **devices)

torch.Size([1, 256, 13, 13])
Now loading the NSD_shared1000 image set and the EVC-OTC voxel set...
Extracting sample feature_maps with torchinfo.
FeatureExtractor Handle for VisionModel
  16 feature maps (-6 duplicates); 1000 inputs
  Memory required for full extraction: 94.103 GB
  Memory usage limiting device set to: cpu
  Memory usage limit currently set to: 82.710 GB
  2 batch(es) required for current memory limit 
   Batch-001: 7 feature maps; 81.425 GB 
   Batch-002: 9 feature maps; 12.678 GB
